## Import the packages we need (Tali)

In [ ]:
import numpy as np #numpy will read the image into an array for source extraction 
import sep #sep is a source exraction and photometry library 

In [ ]:
from astropy.io import fits #flexible image transport system (fits) is a portable file standard used to store images and tables
import matplotlib.pyplot as plt #needed to show resultant images
from matplotlib import rcParams #rcParams allows us to set color bar size, color, shape, origin, axes.

%matplotlib inline 
#the plots will be displayed below the matplotlib commands and will be stored in the notebook

rcParams['figure.figsize'] = [10., 8.] #set the default size of figures for the notebook 

### Reading image into 2-D numpy array (Fili)

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()

### Indexing the image (Fili)

In [ ]:
image_data = hdu_list[0].data

### Checking the image's resolution (Fili)

In [ ]:
print(type(image_data))
print(image_data.shape)

### Show image (Tali)

In [ ]:
plt.imshow(image_data, cmap='gray', origin = 'lower') #shows the image constructed from the array data using a gray color map.
plt.colorbar() #shows the color bar on the right hand side of the image.

## Get additional data for fun! (Tali)

In [ ]:
print('Min:', np.min(image_data)) #prints the minimum value in the data array 
print('Max:', np.max(image_data)) #prints the maximum value in the data array 
print('Mean:', np.mean(image_data)) #prints the mean value in the data array 
print('Stdev:', np.std(image_data)) #prints the standard deviation of the values in the data array 

## Use logNorm to get a better quality image (Tali)

In [ ]:
from matplotlib.colors import LogNorm #lognorm normalizes the given value to a range of 0 to 1 on a logarithmic scale

In [ ]:
 plt.imshow(image_data, cmap='gray', norm=LogNorm(), origin = 'lower') #shows image with gray color map, normalize on a log scale so the bright features show up better 

cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4]) #color bar on the right hand side should have ticks set at 5000, 10000, and 20000
cbar.ax.set_yticklabels(['5,000', '10,000','20,000']) #set the tick labels on the y-axes color bar at 5k, 10k, and 20k.

plt.savefig('fig1.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.

### Using background data (Fili)

#### We're going to take the background data from the image and subtract it from the image displayed above

In [ ]:
new_data = image_data.byteswap(inplace=True).newbyteorder()
bkg = sep.Background(new_data)

In [ ]:
print(bkg.globalback) 
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('fig2hubble.png',bbox_inches='tight', dpi=300)

### Subtract the noise from the image (Fili)

In [ ]:
data_sub = new_data - bkg 

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower') 
plt.colorbar();
plt.savefig('fig3.png',bbox_inches='tight', dpi=300)

### Count the number of objects (Fili)

In [ ]:
len(objects)

## Create image with ellipses (Tali)

In [ ]:
from matplotlib.patches import Ellipse #will allow us to create ellipses around the objects in the image

#plot the background-subtracted image
fig, ax = plt.subplots() #allows combination of multiple plots on a single figure with axes.
m, s = np.mean(data_sub), np.std(data_sub) #computes the average of the data_sub array elements and the standard deviation
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')
#interpolation = 'nearest' means pixels are shown as a square of multiple pixels - works well when small images are scaled up.
#cmap = 'gray' uses gray color map. Could use magma, plasma, inferno, etc..
#vmin = m-s will map the color scale linearly so white is vmax and darjer gray is vmin.


#plot one ellipse for each object
for i in range(len(objects)): #for all i between 0 and 8640 (length of the objects)
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], #width (semi-major axis) of ellipse is 6 times the object. 
                height=6*objects['b'][i], #height (semi-minor axis) of ellipse is 6 times the height.
                angle=objects['theta'][i] * 180. / np.pi) #angle is theta converted to degrees
    e.set_facecolor('none') #ellipses will NOT be filled in
    e.set_edgecolor('red') #the color of the ellipse is red
    ax.add_artist(e) #adds the ellipse subplot to the image subplot for an entire image
plt.savefig('fig4.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.